<a href="https://colab.research.google.com/github/luisosmx/API_connections/blob/main/Sign_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

class AdobeAcrobatAPI:
    def __init__(self, client_id, client_secret, redirect_uri, scopes):
        self.client_id = client_id
        self.client_secret = client_secret
        self.redirect_uri = redirect_uri
        self.scopes = scopes
        self.access_token = None

    def get_authorization_code(self):
        url = "https://secure.echosign.com/public/oauth/v2"

        parameters = {
            "redirect_uri": self.redirect_uri,
            "response_type": "code",
            "client_id": self.client_id,
            "scope": " ".join(self.scopes)
        }

        response = requests.get(url, params=parameters)

        if response.status_code == 200:
            return response.url
        else:
            print("Failed to obtain authorization code.")
            return None

    def get_access_token(self, code):
        url = "https://secure.na1.echosign.com/oauth/v2/token"

        headers = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Cache-Control": "no-cache"
        }
        data = {
            "grant_type": "authorization_code",
            "code": code,
            "client_id": self.client_id,
            "client_secret": self.client_secret,
            "redirect_uri": self.redirect_uri
        }

        response = requests.post(url, headers=headers, data=data)

        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data.get('access_token')
            return self.access_token
        else:
            print("Failed to obtain access token.", response.status_code)
            print("Response:", response.text)
            return None

    def retrieve_library_documents(self):
        url_documentId = "https://api.na4.adobesign.com/api/rest/v6/libraryDocuments"

        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        response = requests.get(url_documentId, headers=headers)

        if response.status_code == 200:
            print("Successful request")
            data = response.json()
            document_list = data.get('libraryDocumentList', [])
            return document_list
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

    def send_agreement(self, library_document_id, email, name_document):
        url_post = "https://api.na4.adobesign.com/api/rest/v6/agreements"

        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Content-Type": "application/json"
        }

        json_data = {
            "fileInfos": [{
                 "libraryDocumentId": library_document_id
            }],
            "name": name_document,
            "participantSetsInfo": [{
                "memberInfos": [{
                    "email": email
                }],
                "order": 1,
                "role": "SIGNER"
            }],
            "signatureType": "ESIGN",
            "state": "IN_PROCESS"
        }

        response = requests.post(url_post, headers=headers, json=json_data)

        if response.status_code == 201:
            data = response.json()
            agreement_id = data.get('id')
            return f"agreementId = {agreement_id}"
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

    def status_agreement(self, agreement_id):
        url_status = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}'

        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        response = requests.get(url_status, headers=headers)

        if response.status_code == 200:
            print("Successful request")
            data = response.json()
            return data
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

    def send_agreement_merge(self, library_document_ids, email, merge_defaults):
        url_post = "https://api.na4.adobesign.com/api/rest/v6/agreements"

        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Content-Type": "application/json"
        }

        file_infos = [{"libraryDocumentId": doc_id} for doc_id in library_document_ids]

        merge_field_infos = []
        for field, value in merge_defaults.items():
            merge_field_infos.append({"defaultValue": value, "fieldName": field})

        json_data = {
            "fileInfos": file_infos,
            "name": "TestAgreement",
            "participantSetsInfo": [{
                "memberInfos": [{"email": email}],
                "order": 1,
                "role": "SIGNER"
            }],
            "signatureType": "ESIGN",
            "state": "IN_PROCESS",
            "mergeFieldInfo": merge_field_infos
        }

        response = requests.post(url_post, headers=headers, json=json_data)

        if response.status_code == 201:
            data = response.json()
            agreement_id = data.get('id')
            return f"agreementId = {agreement_id}"
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None



    def create_reminder(self, recipientParticipantIds):
        url_reminder = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{self.agreement_id}/reminders'

        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        json_data = {
          "recipientParticipantIds": [
            recipientParticipantIds
          ],
          "status": "ACTIVE"
        }

        response = requests.post(url_reminder, headers=headers,  json=json_data)

        if response.status_code == 201:
            print("Successful request")
            data = response.json()
            return data
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

    def retrieves_id(self, agreement_id):

        url_document = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}/documents'

        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        response = requests.get(url_document, headers=headers)

        if response.status_code == 200:
            data = response.json()
            document_id = data['documents'][0]['id']
            print(f"id_document = {document_id}")
        else:
            print(f"Error: {response.status_code} - {response.text}")

    def retrives_file_stream(self, documentId):
        url_document_retrives = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}/documents/{documentId}'

        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        response = requests.get(url_document_retrives, headers=headers)

        if response.status_code == 200:
            print("Successful request")
            with open("Document.pdf", "wb") as f:
                f.write(response.content)
            print("Document downloaded successfully")
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None



client_id = "CBJCHBCAABAA5z5y6_YYaecwliRsbOJke0uGa4pet5Tr"
client_secret = "7x2LZ8kCABcqdUTPgSFR5o01cVF1ED_k"
redirect_uri = "https://oauth.pstmn.io/v1/callback"
scopes = [
    "user_read:account",
    "user_write:account",
    "user_login:account",
    "agreement_read:account",
    "agreement_write:account",
    "agreement_send:account",
    "library_read:account",
    "library_write:account"
]

acrobat_api = AdobeAcrobatAPI(client_id, client_secret, redirect_uri, scopes)


In [ ]:
import requests

url = "https://secure.echosign.com/oauth/v2/token"

payload = 'code=CBNCKBAAHBCAABAA7BOd3nK2GXsA7ViTlPyqGq8Fsh09bk75&client_id=CBJCHBCAABAA5z5y6_YYaecwliRsbOJke0uGa4pet5Tr&client_secret=7x2LZ8kCABcqdUTPgSFR5o01cVF1ED_k&grant_type=authorization_code&redirect_uri=https%3A%2F%2Foauth.pstmn.io%2Fv1%2Fcallback'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Cookie': 'JSESSIONID=BBAABEE8D3F31B5D6157426560756C4D.app-a14; JSESSIONID=BBAABEE8D3F31B5D6157426560756C4D.app-a14'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)
